In [ ]:
%pip install pandas matplotlib seaborn Prophet
%pip install --upgrade pip

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [95]:
def retrieve_and_clean_df(csv_path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df['employment_rate_overall'] = df['employment_rate_overall'].str.replace('%', '')
    df['employment_rate_overall'] = pd.to_numeric(df['employment_rate_overall'], errors='coerce',downcast='float')

    df['employment_rate_ft_perm'] = df['employment_rate_ft_perm'].str.replace('%', '')
    df['employment_rate_ft_perm'] = pd.to_numeric(df['employment_rate_ft_perm'], errors='coerce',downcast='float')

    df['basic_monthly_mean'] = pd.to_numeric(df['basic_monthly_mean'], errors='coerce',downcast='integer')
    df['basic_monthly_median'] = pd.to_numeric(df['basic_monthly_median'], errors='coerce',downcast='integer')
    df['gross_monthly_mean'] = pd.to_numeric(df['gross_monthly_mean'], errors='coerce',downcast='integer')
    df['gross_monthly_median'] = pd.to_numeric(df['gross_monthly_median'], errors='coerce',downcast='integer')
    df['gross_mthly_25_percentile'] = pd.to_numeric(df['gross_mthly_25_percentile'], errors='coerce',downcast='integer')
    df['gross_mthly_75_percentile'] = pd.to_numeric(df['gross_mthly_75_percentile'], errors='coerce',downcast='integer')

    df.loc[df['school'] == 'na', 'school'] = df['university']
    df['degree'] = df['degree'].str.replace('^', '', regex=False).str.strip()
    df['degree'] = df['degree'].str.replace('#', '', regex=False).str.strip()
    df['degree'] = df['degree'].str.replace('\n', '', regex=False).str.strip()

    return df.dropna().reset_index(drop=True)

In [96]:
df = retrieve_and_clean_df('ges.csv')
df.to_csv('GraduateEmploymentSurvey.csv', index=False)

In [97]:
def clean_smu_degree(df: pd.DataFrame) -> pd.DataFrame:
    mask_uni = df['university'] == 'Singapore Management University'

    df.loc[mask_uni, 'school'] = df.loc[mask_uni, 'school'].str.replace('*', '', regex=False).str.strip()
    df.loc[mask_uni, 'degree'] = df.loc[mask_uni, 'degree'].str.replace('#', '', regex=False).str.strip()

    mask_school = df['school'].str.contains('years', regex=False)
    df.loc[mask_uni & mask_school, 'school'] = (
        df.loc[mask_uni & mask_school, 'school'].str.replace('years', 'year', regex=False).str.strip()
    )

    mask_degree1 = df['degree'].str.contains('years', regex=False)
    df.loc[mask_uni & mask_degree1, 'degree'] = (
        df.loc[mask_uni & mask_degree1, 'degree'].str.replace('years', 'year', regex=False).str.strip()
    )

    mask_degree2 = df['degree'].str.contains('(4-year programme)', regex=False)
    df.loc[mask_uni & mask_degree2, 'degree'] = (
        df.loc[mask_uni & mask_degree2, 'degree'].str.replace(' (4-year programme)', '').str.strip()
    )

    mask_degree3 = df['degree'].str.endswith('Cum Laude and above')
    df.loc[mask_uni & mask_degree3, 'degree'] = (
        df.loc[mask_uni & mask_degree3, 'degree'].str.replace('Cum Laude and above', '(Cum Laude and above)', regex= False).str.strip()
    )

    smu_dict = (
        df.loc[~df['degree'].str.contains('(Cum Laude and above)', regex=False), ['school', 'degree']]
        .drop_duplicates().set_index('school')['degree'].to_dict()
    )
    mask_degree4 = df['degree'] == '(Cum Laude and above)'
    df.loc[mask_uni & mask_degree4, 'degree'] = (
        df.loc[mask_uni & mask_degree4, 'school'].map(smu_dict).fillna('').astype(str) + ' (Cum Laude and above)'
    )

    return df

In [98]:
df = clean_smu_degree(df)
df.to_csv('GraduateEmploymentSurvey.csv', index=False)

In [99]:
def clean_degree_name(df: pd.DataFrame) -> pd.DataFrame:
    df["degree"] = df["degree"].str.replace(r'&|And', "and", regex=True)
    df["degree"] = df["degree"].str.replace(r'\bSport\b', "Sports", regex=True)
    df["degree"] = df["degree"].str.replace(r'\bArt\b', "Arts", regex=True)
    df["degree"] = df["degree"].str.replace(r'\bMajor8\b', "Major", regex=True)
    df["degree"] = df["degree"].str.replace(r'\bSciences\b', "Science", regex=True)

    df["degree"] = df["degree"].str.replace('.', '', regex=False)

    # 2. Run your spacing patterns
    # These will now catch the 'and' that used to be a '&'
    forward_pattern = r'(?<=[a-z])(?=[A-Z])|(?<!Cha)(?<=[a-z])(?=(?:and|in|of)(?![a-z]))|(?<! )(?=\()'
    backward_pattern = r'(?<=(?<![a-zA-Z])and)(?=[A-Z])|(?<=(?<![a-zA-Z])in)(?=[A-Z])|(?<=(?<![a-zA-Z])of)(?=[A-Z])'
    df["degree"] = df["degree"].str.replace(forward_pattern, " ", regex=True)
    df["degree"] = df["degree"].str.replace(backward_pattern, " ", regex=True)

    # 3. Handle the specific 'Arts (With' formatting
    pattern = r'((Arts|Science) \()(with|and|With)'
    df["degree"] = df["degree"].str.replace(pattern, r'\1With', regex=True, case=False)

    # 4. Final Cleanup: Remove any double spaces that might have been created
    df["degree"] = df["degree"].str.replace(r'\s+', ' ', regex=True).str.strip()

    return df


In [100]:
df = clean_degree_name(df)
df.to_csv('GraduateEmploymentSurvey.csv', index=False)

In [ ]:
import pandas as pd
from prophet import Prophet
from pandas.tseries.offsets import YearEnd
from datetime import datetime

def get_predictions(df, target_columns, degree_col='degree'):
    current_year = datetime.now().year
    unique_degrees = df[degree_col].unique()
    
    all_new_rows = []

    for degree in unique_degrees:
        # Get all rows for this degree to capture 'other' column values
        degree_df = df[df[degree_col] == degree].copy()
        
        # Identify "other" columns (excluding year, degree, and targets)
        other_cols = [c for c in df.columns if c not in target_columns and c != 'year' and c != degree_col]
        
        # Get the most recent values for the 'other' columns to carry forward
        last_known_metadata = degree_df.sort_values('year').iloc[-1][other_cols]

        # Dictionary to store predictions for this specific degree
        degree_predictions = {}

        for target in target_columns:
            temp_df = degree_df.groupby('year')[target].mean().reset_index()
            last_data_year = temp_df['year'].max()
            periods_to_forecast = max(0, current_year - last_data_year) 
            
            if len(temp_df) < 2 or periods_to_forecast == 0:
                continue

            temp_df.columns = ['ds', 'y']
            temp_df['ds'] = pd.to_datetime(temp_df['ds'], format='%Y') + YearEnd(0)

            model = Prophet(yearly_seasonality=False, changepoint_prior_scale=0.05)
            model.fit(temp_df)
            
            future = model.make_future_dataframe(periods=periods_to_forecast, freq='YE')
            forecast = model.predict(future)
            
            # Keep only the future years
            forecast['year'] = forecast['ds'].dt.year
            new_preds = forecast[forecast['year'] > last_data_year][['year', 'yhat']]
            degree_predictions[target] = new_preds

        # If we have predictions, merge them and add the 'other' columns
        if degree_predictions:
            # Combine multiple targets for this degree
            first_target = list(degree_predictions.keys())[0]
            combined_future = degree_predictions[first_target].rename(columns={'yhat': first_target})
            
            for target in list(degree_predictions.keys())[1:]:
                target_df = degree_predictions[target].rename(columns={'yhat': target})
                combined_future = pd.merge(combined_future, target_df, on='year', how='outer')

            # Add the degree column back
            combined_future[degree_col] = degree
            
            # Carry forward the 'other' metadata columns
            for col in other_cols:
                combined_future[col] = last_known_metadata[col]
            
            all_new_rows.append(combined_future)

    if not all_new_rows:
        return df

    # Combine everything
    final_predictions = pd.concat(all_new_rows, ignore_index=True)

    df['data_source'] = 'actual'
    final_predictions['data_source'] = 'predicted'

    return pd.concat([df, final_predictions], ignore_index=True).sort_values(['year', degree_col])

In [ ]:
numeric_columns = [
    "employment_rate_overall",
    "employment_rate_ft_perm",
    "basic_monthly_mean",
    "basic_monthly_median",
    "gross_monthly_mean",
    "gross_monthly_median",
    "gross_mthly_25_percentile",
    "gross_mthly_75_percentile"
]

prediction_df = get_predictions(df, numeric_columns)
prediction_df.to_csv('GES_with_Predictions.csv', index=False)

21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing
21:25:30 - cmdstanpy - INFO - Chain [1] start processing
21:25:30 - cmdstanpy - INFO - Chain [1] done processing


,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile
0,2014,Singapore Institute of Technology,Trinity College Dublin,Bachelor in Science (Physiotherapy),100.000000,100.000000,3204.000000,3200.000000,3229.000000,3200.000000,3000.000000,3500.000000
1,2015,Singapore Institute of Technology,Trinity College Dublin,Bachelor in Science (Physiotherapy),100.000000,98.300003,3234.000000,3200.000000,3310.000000,3222.000000,3200.000000,3500.000000
2,2016,Singapore Institute of Technology,Trinity College Dublin / Singapore Institute o...,Bachelor in Science (Physiotherapy),100.000000,100.000000,3310.000000,3300.000000,3344.000000,3300.000000,3240.000000,3500.000000
3,2017,Singapore Institute of Technology,Singapore Institute of Technology -Trinity Col...,Bachelor in Science (Physiotherapy),100.000000,96.800003,3483.000000,3350.000000,3489.000000,3400.000000,3300.000000,3500.000000
4,2018,Singapore Institute of Technology,SIT-Trinity College Dublin / Trinity College D...,Bachelor in Science (Physiotherapy),100.000000,100.000000,3258.000000,3300.000000,3276.000000,3300.000000,3000.000000,3600.000000
5,2019,Singapore Institute of Technology,SIT-Trinity College Dublin / Trinity College D...,Bachelor in Science (Physiotherapy),98.300003,98.300003,3479.000000,3400.000000,3498.000000,3400.000000,3400.000000,3620.000000
6,2020,Singapore Institute of Technology,SIT / SIT-Trinity College Dublin / Trinity Col...,Bachelor in Science (Physiotherapy),98.199997,96.300003,3485.000000,3480.000000,3518.000000,3500.000000,3400.000000,3700.000000
7,2022,Singapore Institute of Technology,SIT-Trinity College Dublin,Bachelor in Science (Physiotherapy),98.099998,98.099998,3639.000000,3630.000000,3697.000000,3674.000000,3500.000000,3850.000000
8,2023,Singapore Institute of Technology,SIT-Trinity College Dublin,Bachelor in Science (Physiotherapy),98.028410,97.079747,3660.651053,3687.848864,3695.960674,3729.319942,3532.772219,3927.006715
9,2024,Singapore Institute of Technology,SIT-Trinity College Dublin,Bachelor in Science (Physiotherapy),97.963876,96.819613,3711.782328,3757.216578,3747.416034,3806.250375,3584.580707,4003.558680
